In [1]:
import subprocess as sp
import numpy as np
import pandas as pd
from io import StringIO
import os
import re
import shutil

from utils import *

In [2]:
#global parameters
cudadir = "/usr/common/software/cuda/11.0.167"
homedir = os.path.dirname(os.getcwd())

In [3]:
#input and output dirs
datadirs = ["/global/cfs/cdirs/m1759/yswang/data/conv2d-tf2-noxla"]
outputdir = "/global/cfs/cdirs/m1759/yswang/results/conv2d-tf2-noxla"

# Functions

In [4]:
def transpose_frame(df_metrics):
    #Copy the profile frame to make sure not to overwrite it and potentially read it in again if we screwed it up
    selectkeys = ["ID", "Precision", "Network Name", "Data Format", "Input Shape", "Kernel Shape", "Stride Size", "Batch Size", "Pass", "Name"]
    resultkeys = ["Precision", "Network Name", "Data Format", "Input Shape", "Kernel Shape", "Stride Size", "Batch Size", "Pass", "Name"]

    #as metricdf use df_summary
    metricdf = df_metrics.copy()
    #metricdf.sort_values(by=selectkeys,inplace=True)
    #metricdf.reset_index(drop=True, inplace=True)

    #remove the calibration
    metricdf = metricdf[metricdf["Pass"] != "calibrate"]
    profiledf = pd.DataFrame(columns=selectkeys)
    
    ####### Get timing information
    ### CUDA Time
    # get cycles
    metricname = "CUDA Cycles"
    cyclesdf = metricdf.loc[(metricdf["Metric Name"]=="sm__cycles_elapsed") & (metricdf["Metric Type"]=="total"),
                           selectkeys+["Metric Unit", "Metric Value"]].reset_index(drop=True).sort_values(by=selectkeys).rename(columns={"Metric Value": metricname}).copy()
    # get rates
    metricname = "CUDA Rates"
    ratesdf = metricdf.loc[(metricdf["Metric Name"]=="sm__cycles_elapsed") & (metricdf["Metric Type"]=="rate"),
                           selectkeys+["Metric Unit", "Metric Value"]].reset_index(drop=True).sort_values(by=selectkeys).rename(columns={"Metric Value": metricname}).copy()
    # check consistency
    if not cyclesdf[['ID', 'Name']].equals(ratesdf[['ID', 'Name']]):
        raise ValueError("CUDA Time data not consistent")
    # adjust metric unit
    ratesdf.loc[ratesdf["Metric Unit"].str.contains("cycle/nsecond"), ["CUDA Rates"]] *= 1e9
    # manual merge and compute CUDA Time
    cyclesdf["CUDA Rates"] = list(ratesdf["CUDA Rates"])
    cyclesdf["CUDA Time"] = cyclesdf["CUDA Cycles"] / cyclesdf["CUDA Rates"]
    # merge with output
    profiledf = cyclesdf[selectkeys+['CUDA Time']].copy()
    
    ### Combine
    del profiledf['ID']
    del metricdf['ID']
    profiledf['Invocations'] = 1
    profiledf = profiledf.groupby(resultkeys).sum().reset_index()
        
    ####### Get number of FLOPs
    
    ### FMA FLOPs = number of FMA instructions x 2
    metricdf.loc[metricdf["Metric Name"].str.contains("fma"), ["Metric Value"]] *= 2
    
    ### FP32 FLOPs
    metrics = ['sm__sass_thread_inst_executed_op_fadd_pred_on',
               'sm__sass_thread_inst_executed_op_ffma_pred_on',
               'sm__sass_thread_inst_executed_op_fmul_pred_on']
    tmpdf = metricdf.loc[ metricdf["Metric Name"].isin(metrics), resultkeys+["Metric Value"] ].copy()
    tmpdf = tmpdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "FP32 FLOPs"})
    # merge
    profiledf = profiledf.merge(tmpdf[resultkeys+["FP32 FLOPs"]], on=resultkeys, how="inner")
    
    ### FP16 FLOPs
    metrics = ['sm__sass_thread_inst_executed_op_hadd_pred_on',
               'sm__sass_thread_inst_executed_op_hfma_pred_on',
               'sm__sass_thread_inst_executed_op_hmul_pred_on']
    tmpdf = metricdf.loc[ metricdf["Metric Name"].isin(metrics), resultkeys+["Metric Value"] ].copy()
    tmpdf = tmpdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "FP16 FLOPs"})
    # merge
    profiledf = profiledf.merge(tmpdf[resultkeys+["FP16 FLOPs"]], on=resultkeys, how="inner")
    
    ### TC FLOPs
    tmpdf = metricdf.loc[ metricdf["Metric Name"].str.contains("sm__inst_executed_pipe_tensor"), resultkeys+["Metric Value"] ].copy()
    tmpdf = tmpdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "TC FLOPs"})
    tmpdf["TC FLOPs"] = 512 * tmpdf["TC FLOPs"]
    # merge
    profiledf = profiledf.merge(tmpdf[resultkeys+["TC FLOPs"]], on=resultkeys, how="inner")

    ### Total FLOPs
    profiledf["FLOPs"] = profiledf["FP32 FLOPs"] + profiledf["FP16 FLOPs"] + profiledf["TC FLOPs"] #+ metricdf["FP64 FLOPs"]
        
    ### FLOPs fractions
    #profiledf["FP64 FLOPs Fraction"] = profiledf["FP64 FLOPs"]/profiledf["FLOPs"]
    profiledf["FP32 FLOPs Fraction"] = profiledf["FP32 FLOPs"]/profiledf["FLOPs"]
    profiledf["FP16 FLOPs Fraction"] = profiledf["FP16 FLOPs"]/profiledf["FLOPs"]
    profiledf["TC FLOPs Fraction"]   = profiledf["TC FLOPs"]/profiledf["FLOPs"]
    profiledf = profiledf.fillna(0.)

    ####### Get number of bytes
    
    # adjust metric unit
    metricdf.loc[(metricdf["Metric Unit"]=="Kbyte"), ["Metric Value"]] *= 1e3
    metricdf.loc[(metricdf["Metric Unit"]=="Mbyte"), ["Metric Value"]] *= 1e6
    metricdf.loc[(metricdf["Metric Unit"]=="Gbyte"), ["Metric Value"]] *= 1e9
    
    ### L1 Bytes
    #project out
    l1df = metricdf.loc[metricdf["Metric Name"].str.contains("l1tex__t_bytes"), selectkeys+["Metric Value"] ].copy()
    l1df = l1df.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "L1 Bytes"})
    # merge
    profiledf = profiledf.merge(l1df[resultkeys+["L1 Bytes"]], on=resultkeys, how="inner")
    
    ### L2 Bytes
    #project out
    l2df = metricdf.loc[metricdf["Metric Name"].str.contains("lts__t_bytes"), selectkeys+["Metric Value"] ].copy()
    l2df = l2df.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "L2 Bytes"})
    # merge
    profiledf = profiledf.merge(l2df[resultkeys+["L2 Bytes"]], on=resultkeys, how="inner")
    
    ### DRAM Bytes
    #project out
    dramdf = metricdf.loc[metricdf["Metric Name"].str.contains("dram__bytes"), selectkeys+["Metric Value"] ].copy()
    dramdf = dramdf.groupby(resultkeys).sum().reset_index().rename(columns={"Metric Value": "DRAM Bytes"})
    # merge
    profiledf = profiledf.merge(dramdf[resultkeys+["DRAM Bytes"]], on=resultkeys, how="inner")
    
    ### Get performance
    profiledf["Performance GFlop/s"] = profiledf["FLOPs"] / (profiledf["CUDA Time"]*10**9)

    ### Get AI
    # L1
    profiledf["L1 AI"]        = profiledf["FLOPs"]      / profiledf["L1 Bytes"]
    profiledf["FP32 L1 AI"]   = profiledf["FP32 FLOPs"] / profiledf["L1 Bytes"]
    profiledf["FP16 L1 AI"]   = profiledf["FP16 FLOPs"] / profiledf["L1 Bytes"]
    profiledf["TC L1 AI"]     = profiledf["TC FLOPs"]   / profiledf["L1 Bytes"]
    # L2
    profiledf["L2 AI"]        = profiledf["FLOPs"]      / profiledf["L2 Bytes"]
    profiledf["FP32 L2 AI"]   = profiledf["FP32 FLOPs"] / profiledf["L2 Bytes"]
    profiledf["FP16 L2 AI"]   = profiledf["FP16 FLOPs"] / profiledf["L2 Bytes"]
    profiledf["TC L2 AI"]     = profiledf["TC FLOPs"]   / profiledf["L2 Bytes"]
    # DRAM
    profiledf["DRAM AI"]      = profiledf["FLOPs"]      / profiledf["DRAM Bytes"]
    profiledf["FP32 DRAM AI"] = profiledf["FP32 FLOPs"] / profiledf["DRAM Bytes"]
    profiledf["FP16 DRAM AI"] = profiledf["FP16 FLOPs"] / profiledf["DRAM Bytes"]
    profiledf["TC DRAM AI"]   = profiledf["TC FLOPs"]   / profiledf["DRAM Bytes"]
    
    ### Cleanup
    profiledf.sort_values(by=resultkeys).reset_index(drop=True, inplace=True)

    return profiledf

# Import Data

In [5]:
#get all the files
files = []
for datadir in datadirs:
    files += [ os.path.join(datadir,x) for x in os.listdir(datadir) if ((os.path.splitext(x)[-1] == ".ncu-rep"))]

#recs
records = []

#build feature list:
for path in files:
    
    #filename
    file = os.path.basename(path)
    
    #path
    path = os.path.dirname(path)
    
    #splitup
    splt = file.split(".")
    
    prefix = ".".join(splt[0:-1])
    
    #append to records
    records.append({"prefix": prefix, "file": os.path.join(path, file)})

#put in df
recorddf = pd.DataFrame(records).sort_values(["prefix"])
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#display(recorddf["prefix"])

In [6]:
#group by prefixes and files
all_prefixes = set([x.split(".pass")[0] for x in recorddf["prefix"]])
xla_list = set([x.split(".")[-1] for x in recorddf["prefix"]])
if (len(xla_list) != 1):
    raise RuntimeError("too many xla options.")
xla = xla_list.pop()

all_passes = set([x.split(".pass_")[1].replace(".pass_","") for x in recorddf["prefix"].unique()])
all_passes = set([x.split(".")[0] for x in all_passes])

#metrics
df_profiles = []

for pref in all_prefixes:
    
    #set empty lists
    df_times = []
    df_timeline = []
    df_summary = []
    
    #print prefix
    #print(pref)
    
    #loop over passes
    df_times = []
    df_metrics = []
    for pas in all_passes:
        
        #project frame
        files = recorddf.loc[ recorddf["prefix"] == pref + ".pass_" + pas + "." + xla, "file" ].values
        
        #project the invididual files
        metricfile = [x for x in files if x.endswith(".ncu-rep")][0]
        print(metricfile)
            
        #get the parameters from the filename
        parameters = parse_filename_nsight(os.path.basename(metricfile))
            
        #metrics
        metricdf = import_nsight_metric(metricfile, cuda_dir=cudadir)
        for key in parameters:
            metricdf[key] = parameters[key]

        #fuse read/write metrics together:
        unique_metrics = metricdf["Metric Name"].unique()
        
        unique_metrics = set([x.replace(".sum","").replace(".per_second","").replace(".avg","") for x in unique_metrics])
        #add the metric type
        metricdf["Metric Type"] = "total"
        #rate
        metricdf.loc[ metricdf[ "Metric Name" ].str.contains(".per_second"), "Metric Type" ] = "rate"
                
        for metric in unique_metrics:
            metricdf.loc[ metricdf[ "Metric Name"].str.startswith(metric), "Metric Name" ] = metric
                
        #append to DF:
        df_metrics.append(metricdf)
    
    metricdf = pd.concat(df_metrics)
    
    #compute the profile
    profiledf = transpose_frame(metricdf)
    df_profiles.append(profiledf)

#concat everything
profiledf = pd.concat(df_profiles)
profiledf.reset_index(drop=True, inplace=True)

/global/cfs/cdirs/m1759/yswang/data/conv2d-tf2-noxla/tf2.name_ResNet50-2.batch_16.input_112x112x64.kernel_9x9x64x64.stride_2.data_NHWC.fp32.pass_backward.noxla.ncu-rep
/global/cfs/cdirs/m1759/yswang/data/conv2d-tf2-noxla/tf2.name_ResNet50-2.batch_16.input_112x112x64.kernel_9x9x64x64.stride_2.data_NHWC.fp32.pass_forward.noxla.ncu-rep


/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


/global/cfs/cdirs/m1759/yswang/data/conv2d-tf2-noxla/tf2.name_ResNet50-2.batch_16.input_112x112x64.kernel_9x9x64x64.stride_2.data_NHWC.fp32.pass_calibrate.noxla.ncu-rep
/global/cfs/cdirs/m1759/yswang/data/conv2d-tf2-noxla/tf2.name_ResNet50-2.batch_16.input_112x112x64.kernel_3x3x64x256.stride_2.data_NHWC.fp16.pass_backward.noxla.ncu-rep
/global/cfs/cdirs/m1759/yswang/data/conv2d-tf2-noxla/tf2.name_ResNet50-2.batch_16.input_112x112x64.kernel_3x3x64x256.stride_2.data_NHWC.fp16.pass_forward.noxla.ncu-rep
/global/cfs/cdirs/m1759/yswang/data/conv2d-tf2-noxla/tf2.name_ResNet50-2.batch_16.input_112x112x64.kernel_3x3x64x256.stride_2.data_NHWC.fp16.pass_calibrate.noxla.ncu-rep
/global/cfs/cdirs/m1759/yswang/data/conv2d-tf2-noxla/tf2.name_ResNet50-2.batch_64.input_112x112x64.kernel_3x3x64x64.stride_2.data_NHWC.fp16.pass_backward.noxla.ncu-rep
/global/cfs/cdirs/m1759/yswang/data/conv2d-tf2-noxla/tf2.name_ResNet50-2.batch_64.input_112x112x64.kernel_3x3x64x64.stride_2.data_NHWC.fp16.pass_forward.nox

In [7]:
display(profiledf)

,Precision,Network Name,Data Format,Input Shape,Kernel Shape,Stride Size,Batch Size,Pass,Name,CUDA Time,...,FP16 L1 AI,TC L1 AI,L2 AI,FP32 L2 AI,FP16 L2 AI,TC L2 AI,DRAM AI,FP32 DRAM AI,FP16 DRAM AI,TC DRAM AI
0,FP32,ResNet50-2,NHWC,112x112x64,9x9x64x64,2,16,backward,cudnn::gemm::computeOffsetsKernel(cudnn::gemm:...,0.000003,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
1,FP32,ResNet50-2,NHWC,112x112x64,9x9x64x64,2,16,backward,cudnn::gemm::computeWgradBOffsetsKernel(cudnn:...,0.000007,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
2,FP32,ResNet50-2,NHWC,112x112x64,9x9x64x64,2,16,backward,cudnn::gemm::computeWgradSplitKOffsetsKernel(c...,0.000007,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
3,FP32,ResNet50-2,NHWC,112x112x64,9x9x64x64,2,16,backward,void Eigen::internal::EigenMetaKernel<Eigen::T...,0.000003,...,0.000000,0.000000,0.000120,0.00012,0.000000,0.00000,0.000082,0.000082,0.000000,0.000000
4,FP32,ResNet50-2,NHWC,112x112x64,9x9x64x64,2,16,backward,void Eigen::internal::EigenMetaKernel<Eigen::T...,0.000045,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,FP16,ResNet50-2,NHWC,112x112x64,7x7x64x64,2,16,forward,void Eigen::internal::EigenMetaKernel<Eigen::T...,0.000065,...,0.111116,0.000000,0.124952,0.00000,0.124952,0.00000,0.132999,0.000000,0.132999,0.000000
456,FP16,ResNet50-2,NHWC,112x112x64,7x7x64x64,2,16,forward,void tensorflow::functor::FillPhiloxRandomKern...,0.000099,...,0.128112,0.000000,0.105621,0.00000,0.105621,0.00000,0.279743,0.000000,0.279743,0.000000
457,FP16,ResNet50-2,NHWC,112x112x64,7x7x64x64,2,16,forward,void tensorflow::functor::PadInputCustomKernel...,0.000209,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
458,FP16,ResNet50-2,NHWC,112x112x64,7x7x64x64,2,16,forward,void tensorflow::functor::ShuffleInTensor3Simp...,0.000008,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


# Compute AI Results

In [8]:
#sum over all kernels
combinedselectkeys = ["Precision", "Network Name", "Data Format", "Input Shape", "Kernel Shape", "Stride Size", \
                     "Batch Size", "Pass"]

#copy profiledf
combineddf = profiledf.copy()

#sum up
combineddf = combineddf.groupby(by=combinedselectkeys).sum()


#the flop fractions need to be recomputed
combineddf["FP32 FLOPs Fraction"] = combineddf["FP32 FLOPs"] / combineddf["FLOPs"]
combineddf["FP16 FLOPs Fraction"] = combineddf["FP16 FLOPs"] / combineddf["FLOPs"]
combineddf["TC FLOPs Fraction"]   = combineddf["TC FLOPs"]   / combineddf["FLOPs"]

### Get performance
combineddf["Performance GFlop/s"]      = combineddf["FLOPs"]      / (combineddf["CUDA Time"]*10**9)


### Get AI
# L1
combineddf["L1 AI"]        = combineddf["FLOPs"]      / combineddf["L1 Bytes"]
combineddf["FP32 L1 AI"]   = combineddf["FP32 FLOPs"] / combineddf["L1 Bytes"]
combineddf["FP16 L1 AI"]   = combineddf["FP16 FLOPs"] / combineddf["L1 Bytes"]
combineddf["TC L1 AI"]     = combineddf["TC FLOPs"]   / combineddf["L1 Bytes"]
# L2
combineddf["L2 AI"]        = combineddf["FLOPs"]      / combineddf["L2 Bytes"]
combineddf["FP32 L2 AI"]   = combineddf["FP32 FLOPs"] / combineddf["L2 Bytes"]
combineddf["FP16 L2 AI"]   = combineddf["FP16 FLOPs"] / combineddf["L2 Bytes"]
combineddf["TC L2 AI"]     = combineddf["TC FLOPs"]   / combineddf["L2 Bytes"]
# DRAM
combineddf["DRAM AI"]      = combineddf["FLOPs"]      / combineddf["DRAM Bytes"]
combineddf["FP32 DRAM AI"] = combineddf["FP32 FLOPs"] / combineddf["DRAM Bytes"]
combineddf["FP16 DRAM AI"] = combineddf["FP16 FLOPs"] / combineddf["DRAM Bytes"]
combineddf["TC DRAM AI"]   = combineddf["TC FLOPs"]   / combineddf["DRAM Bytes"]

combineddf.sort_values(by=combinedselectkeys).reset_index(drop=True, inplace=True)

In [9]:
display(combineddf)

CUDA Time  \
Precision Network Name Data Format Input Shape Kernel Shape Stride Size Batch Size Pass                  
FP16      ResNet50-2   NHWC        112x112x64  3x3x64x128   2           16         backward   0.002754   
                                                                                   forward    0.000595   
                                               3x3x64x256   2           16         backward   0.003384   
                                                                                   forward    0.000685   
                                               3x3x64x512   2           16         backward   0.005005   
                                                                                   forward    0.000899   
                                               3x3x64x64    1           16         backward   0.001680   
                                                                                   forward    0.000490   
                                                            2           16         backward   0.003303   
                                                                                   forward    0.000550   
                                                                        32         backward   0.004742   
                                                                                   forward    0.001070   
                                                                        64         backward   0.009181   
                                                                                   forward    0.002101   
                                                            3           16         backward   0.001536   
                                                                                   forward    0.000727   
                                               7x7x64x64    2           16         backward   0.003502   
                                                                                   forward    0.000963   
                                               9x9x64x64    2           16         backward   0.011835   
                                                                                   forward    0.001278   
FP32      ResNet50-2   NHWC        112x112x64  3x3x64x128   2           16         backward   0.007741   
                                                                                   forward    0.001552   
                                               3x3x64x256   2           16         backward   0.010945   
                                                                                   forward    0.001997   
                                               3x3x64x512   2           16         backward   0.017935   
                                                                                   forward    0.003269   
                                               3x3x64x64    1           16         backward   0.007541   
                                                                                   forward    0.001488   
                                                            2           16         backward   0.006079   
                                                                                   forward    0.001201   
                                                                        32         backward   0.012126   
                                                                                   forward    0.002053   
                                                                        64         backward   0.023160   
                                                                                   forward    0.004176   
                                                            3           16         backward   0.003047   
                                                                                   forward    0.000712   
                                               7x7x64x64    2    

In [10]:
display(combineddf[["L2 AI", "L1 AI"]])

L2 AI  \
Precision Network Name Data Format Input Shape Kernel Shape Stride Size Batch Size Pass                  
FP16      ResNet50-2   NHWC        112x112x64  3x3x64x128   2           16         backward  40.352514   
                                                                                   forward   18.212400   
                                               3x3x64x256   2           16         backward  34.434355   
                                                                                   forward   27.675143   
                                               3x3x64x512   2           16         backward  61.027476   
                                                                                   forward   43.656891   
                                               3x3x64x64    1           16         backward  34.670225   
                                                                                   forward   27.620078   
                                                            2           16         backward  18.427881   
                                                                                   forward    8.876985   
                                                                        32         backward  37.270887   
                                                                                   forward    8.788506   
                                                                        64         backward  37.617604   
                                                                                   forward    8.764587   
                                                            3           16         backward  25.069337   
                                                                                   forward    4.682897   
                                               7x7x64x64    2           16         backward  37.183309   
                                                                                   forward   29.697489   
                                               9x9x64x64    2           16         backward  33.249674   
                                                                                   forward   35.889567   
FP32      ResNet50-2   NHWC        112x112x64  3x3x64x128   2           16         backward   6.703633   
                                                                                   forward    8.097039   
                                               3x3x64x256   2           16         backward  10.425271   
                                                                                   forward   13.215194   
                                               3x3x64x512   2           16         backward  13.488322   
                                                                                   forward   16.723967   
                                               3x3x64x64    1           16         backward   8.064909   
                                                                                   forward    3.416138   
                                                            2           16         backward   3.825667   
                                                                                   forward    4.184170   
                                                                        32         backward   4.257824   
                                                                                   forward    5.834382   
                                                                        64         backward   4.250433   
                                                                                   forward    5.658899   
                                                            3           16         backward   3.451891   
                                                                                   forward    2.958275   
                                               7x7x64x64    2        

# Export Data

In [11]:
profiledf.to_csv(os.path.join(outputdir,"full_profile.csv"))
combineddf.to_csv(os.path.join(outputdir,"combined_profile.csv"))